In [1]:
%matplotlib widget

from pathlib import Path

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from carwatch_analysis.saliva_helper import import_cortisol_raw, import_cortisol_features, analysis_saliva_features

import biopsykit as bp
from biopsykit.stats import StatsPipeline

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

from pingouin import mixed_anova, rm_anova

C:\Users\luca_\Anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.12, the latest is 0.4.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## Read files

In [2]:
export_path = Path("../..").joinpath("exports")

In [3]:
cort_feat = pd.read_csv(export_path.joinpath('cortisol_features_app_cleaned.csv'))
cort_feat.head()

,subject,condition,chronotype,MEQ,night,wakeup_source,weekend,date,wakeup_hour,log_mode,saliva_feature,cortisol
0,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,app,cortisol_auc_g,697.398333
1,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,app,cortisol_auc_i,339.698333
2,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,app,cortisol_max_inc,7.030000
3,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,app,cortisol_slopeS0S3,0.140430
4,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,app,cortisol_slopeS0S4,0.073973


In [4]:
cort_samples = pd.read_csv(export_path.joinpath('cortisol_samples_app_cleaned.csv'))
cort_samples.head()

,subject,condition,chronotype,MEQ,night,wakeup_source,weekend,date,wakeup_hour,sample,cortisol,log_mode,time
0,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S0,5.88,self_report,0.0
1,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S1,12.54,self_report,15.0
2,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S2,12.91,self_report,30.0
3,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S3,12.30,self_report,45.0
4,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S4,10.38,self_report,60.0


### Number of samples (nights)

In [5]:
count = pd.DataFrame(cort_feat.groupby('log_mode').size()/5)
count.rename(columns={0:'number of samples'}, inplace=True)
count

,number of samples
log_mode,
app,70.0
naive,70.0
self_report,70.0


### Boxplots of differences

In [6]:
cort_samples["time"].loc[cort_samples["sample"]=="S1"]=cort_samples.loc[cort_samples["sample"]=="S1"].apply(lambda row: row["time"]-15, axis=1)
cort_samples["time"].loc[cort_samples["sample"]=="S2"]=cort_samples.loc[cort_samples["sample"]=="S2"].apply(lambda row: row["time"]-30, axis=1)
cort_samples["time"].loc[cort_samples["sample"]=="S3"]=cort_samples.loc[cort_samples["sample"]=="S3"].apply(lambda row: row["time"]-45, axis=1)
cort_samples["time"].loc[cort_samples["sample"]=="S4"]=cort_samples.loc[cort_samples["sample"]=="S4"].apply(lambda row: row["time"]-60, axis=1)

In [7]:
cort_samples.head()

,subject,condition,chronotype,MEQ,night,wakeup_source,weekend,date,wakeup_hour,sample,cortisol,log_mode,time
0,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S0,5.88,self_report,0.0
1,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S1,12.54,self_report,0.0
2,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S2,12.91,self_report,0.0
3,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S3,12.30,self_report,0.0
4,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S4,10.38,self_report,0.0


In [17]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.sharey(ax2)

sns.boxplot(x='sample', y='time', data=cort_samples.loc[cort_samples['log_mode']=='self_report'], ax=ax1)
ax1.set_title('self_report')
sns.boxplot(x='sample', y='time', data=cort_samples.loc[cort_samples['log_mode']=='app'], ax= ax2)
ax2.set_title('app')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'app')

## Statistical Analysis

In [6]:
steps = [("prep", "normality"), ("prep", "equal_var"), ("test", "rm_anova"), ("posthoc", "pairwise_ttests")]
params = {"groupby": "saliva_feature", "dv": "cortisol", "within": "log_mode", "subject": "subject", "padjust": "bonf"}
pipeline = StatsPipeline(steps=steps, params=params)

pipeline.apply(cort_feat)
pipeline.display_results()

C:\Users\luca_\Anaconda3\lib\site-packages\pingouin\distribution.py:950: RuntimeWarning: divide by zero encountered in double_scalars
  W = np.product(eig) / (eig.sum() / d)**d
C:\Users\luca_\Anaconda3\lib\site-packages\pingouin\distribution.py:950: RuntimeWarning: divide by zero encountered in double_scalars
  W = np.product(eig) / (eig.sum() / d)**d
C:\Users\luca_\Anaconda3\lib\site-packages\pingouin\distribution.py:950: RuntimeWarning: divide by zero encountered in double_scalars
  W = np.product(eig) / (eig.sum() / d)**d
C:\Users\luca_\Anaconda3\lib\site-packages\pingouin\distribution.py:950: RuntimeWarning: divide by zero encountered in double_scalars
  W = np.product(eig) / (eig.sum() / d)**d
C:\Users\luca_\Anaconda3\lib\site-packages\pingouin\distribution.py:950: RuntimeWarning: divide by zero encountered in double_scalars
  W = np.product(eig) / (eig.sum() / d)**d
C:\Users\luca_\Anaconda3\lib\site-packages\pingouin\parametric.py:222: UserWarning: x and y are equals. Cannot comp

<font size="3"><b> Overview </b></font>

,groupby,dv,within,subject,padjust
parameter,saliva_feature,cortisol,log_mode,subject,bonf


,prep,prep,test,posthoc
parameter,normality,equal_var,rm_anova,pairwise_ttests


<font size="3"><b> Preparatory Analysis </b></font>

...

**Repeated-measurement ANOVA**

,,Source,ddof1,ddof2,F,p-unc,p-GG-corr,np2,eps,sphericity,W-spher,p-spher
saliva_feature,,,,,,,,,,,,
cortisol_auc_g,0,log_mode,2,66,6.244827e+00,0.003282,0.016825,1.591248e-01,0.513516,False,0.052639,3.475021e-21
cortisol_auc_i,0,log_mode,2,66,3.300519e+00,0.043036,0.077915,9.092210e-02,0.504914,False,0.019465,4.246773e-28
cortisol_cini,0,log_mode,2,66,-2.335632e-14,1.000000,NaN,-7.077672e-16,-0.000000,NaN,NaN,NaN
cortisol_cmax,0,log_mode,2,66,1.051034e-13,1.000000,NaN,3.184952e-15,0.000000,NaN,NaN,NaN
cortisol_max_inc,0,log_mode,2,66,1.313793e-14,1.000000,NaN,3.981190e-16,0.000000,NaN,NaN,NaN
cortisol_slopeS0S3,0,log_mode,2,66,3.090440e+00,0.052122,NaN,8.563044e-02,0.508587,NaN,NaN,NaN
cortisol_slopeS0S4,0,log_mode,2,66,2.537518e+00,0.086753,NaN,7.140391e-02,0.504906,NaN,NaN,NaN


<font size="3"><b> Post-Hoc Analysis </b></font>

**Pairwise t-Tests**

Contrast      A            B  Paired  Parametric  \
saliva_feature                                                           
cortisol_auc_g     0  log_mode    app        naive    True        True   
                   1  log_mode    app  self_report    True        True   
                   2  log_mode  naive  self_report    True        True   
cortisol_auc_i     0  log_mode    app        naive    True        True   
                   1  log_mode    app  self_report    True        True   
                   2  log_mode  naive  self_report    True        True   
cortisol_cini      0  log_mode    app        naive    True        True   
                   1  log_mode    app  self_report    True        True   
                   2  log_mode  naive  self_report    True        True   
cortisol_cmax      0  log_mode    app        naive    True        True   
                   1  log_mode    app  self_report    True        True   
                   2  log_mode  naive  self_report    True        True   
cortisol_max_inc   0  log_mode    app        naive    True        True   
                   1  log_mode    app  self_report    True        True   
                   2  log_mode  naive  self_report    True        True   
cortisol_slopeS0S3 0  log_mode    app        naive    True        True   
                   1  log_mode    app  self_report    True        True   
                   2  log_mode  naive  self_report    True        True   
cortisol_slopeS0S4 0  log_mode    app        naive    True        True   
                   1  log_mode    app  self_report    True        True   
                   2  log_mode  naive  self_report    True        True   

                             T   dof       Tail     p-unc    p-corr p-adjust  \
saliva_feature                                                                 
cortisol_auc_g     0  2.718089  33.0  two-sided  0.010381  0.031143     bonf   
                   1  2.203704  33.0  two-sided  0.034632  0.103895     bonf   
                   2 -4.137401  33.0  two-sided  0.000227  0.000682     bonf   
cortisol_auc_i     0  1.882450  33.0  two-sided  0.068617  0.205852     bonf   
                   1  1.742188  33.0  two-sided  0.090790  0.272370     bonf   
                   2 -2.173626  33.0  two-sided  0.037010  0.111029     bonf   
cortisol_cini      0       NaN  33.0  two-sided       NaN       NaN     bonf   
                   1       NaN  33.0  two-sided       NaN       NaN     bonf   
                   2       NaN  33.0  two-sided       NaN       NaN     bonf   
cortisol_cmax      0       NaN  33.0  two-sided       NaN       NaN     bonf   
                   1       NaN  33.0  two-sided       NaN       NaN     bonf   
                   2       NaN  33.0  two-sided       NaN       NaN     bonf   
cortisol_max_inc   0       NaN  33.0  two-sided       NaN       NaN     bonf   
                   1       NaN  33.0  two-sided       NaN       NaN     bonf   
                   2       NaN  33.0  two-sided       NaN       NaN     bonf   
cortisol_slopeS0S3 0 -1.753674  33.0  two-sided  0.088771  0.266314     bonf   
                   1 -1.773735  33.0  two-sided  0.085336  0.256008     bonf   
                   2  0.445027  33.0  two-sided  0.659206  1.000000     bonf   
cortisol_slopeS0S4 0 -1.536008  33.0  two-sided  0.134070  0.402210     bonf   
                   1 -1.665934  33.0  two-sided  0.105193  0.315578     bonf   
                   2 -0.133542  33.0  two-sided  0.894575  1.000000     bonf   

                         BF10    hedges  
saliva_feature                           
cortisol_auc_g     0    4.181  0.159276  
                   1    1.533  0.126695  
                   2  120.259 -0.035248  
cortisol_auc_i     0    0.888  0.077945  
                   1    0.715  0.070356  
                   2    1.453 -0.008057  
cortisol_cini      0      nan  0.000000  
                   1      nan  0.000000  
                   2      nan  0.000000  
cortisol_cmax   